In [3]:
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

## Create Datasets (split into train, test, val)

In [6]:
df = pd.read_csv("../data/metadata.csv")

In [10]:
df.shape

(2298, 26)

In [11]:
new_df = df[['img_id', 'diagnostic']]

In [13]:
# edit file path name
new_df['img_id'] = "../imgs_part_1/" + new_df['img_id'].astype(str)

C:\Users\surfd\AppData\Local\Temp\ipykernel_18136\1488525695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['img_id'] = "../imgs_part_1/" + new_df['img_id'].astype(str)


In [54]:
# edit skin disease names into numeric classes
mapping = {'BCC':0, 'MEL':1, 'SCC':2 , 'ACK':3, 'NEV':4, 'SEK':5}
new_df['class'] = new_df['diagnostic'].map(mapping)
new_df = new_df.drop(['diagnostic'], axis=1)
#new_df = new_df[['img_id', 'class', 'diagnostic']]

In [59]:
# split into training (60%), testing (20%), and validation data (20%)
# train -- 1378, test -- 460, val -- 460
train_df = new_df.iloc[:1378,:]
test_df = new_df.iloc[1378:1838,:]
val_df = new_df.iloc[1838:2298,:]

train_df.to_csv("../data/train.csv", index=False)
test_df.to_csv("../data/test.csv", index=False)
val_df.to_csv("../data/val.csv", index=False)

## Set up to use Efficient Net model

In [70]:
# setup custom dataset

class SkinCancerDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        image = Image.open(img_path).convert("RGB")
        label = int(self.data.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

In [71]:
# augment data / resize

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [72]:
# load data with dataloader

train_dataset = SkinCancerDataset(csv_file='../data/train.csv', transform=train_transforms)
val_dataset = SkinCancerDataset(csv_file='../data/val.csv', transform=val_transforms)
test_dataset = SkinCancerDataset(csv_file='../data/test.csv', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [78]:
# load EfficientNet and modify the classifier layer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(pretrained=True)
num_classes = len(train_dataset.data['class'].unique())
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

## Test base model

In [82]:
from tqdm import tqdm
import torch.nn.functional as F

# run model
model.eval()

# make predictions
val_preds, val_labels = [], []
val_probs = []  # To store predicted probabilities for AUC

with torch.no_grad():
    for images, labels in tqdm(val_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        
        # Get the predicted class for each image (for accuracy and f1)
        preds = outputs.argmax(dim=1)
        
        # Get predicted probabilities for AUC (softmax output)
        probs = F.softmax(outputs, dim=1)
        
        # Store predictions and true labels
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())
        
        # Store probabilities (for AUC)
        val_probs.extend(probs.cpu().numpy())

# calculate metrics
val_accuracy = accuracy_score(val_labels, val_preds)
val_f1 = f1_score(val_labels, val_preds, average='macro')

# For multi-class AUC calculation, we need to use predicted probabilities
val_auc = roc_auc_score(val_labels, val_probs, average='macro', multi_class='ovr')

print(f'Validation Accuracy: {val_accuracy:.4f}')
print(f'Validation F1 Score: {val_f1:.4f}')
print(f'Validation AUC: {val_auc:.4f}')

100%|██████████| 15/15 [00:32<00:00,  2.16s/it]

Validation Accuracy: 0.1435
Validation F1 Score: 0.1207
Validation AUC: 0.4970


## Fine-tuning

In [79]:
# freeze earlier layers / only fine tune the bottom layer
for param in model.features.parameters():
    param.requires_grad = False

In [80]:
# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [76]:
# Training and Validation Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            print(images)
            print(labels)
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = outputs.argmax(dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, '
              f'Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}')

        # Save the best model based on validation accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')

In [77]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

this is images and labels 

tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
  

KeyboardInterrupt: 